# Fast Personalized Pagerank Implementation (Moler PageRank)
I needed a fast PageRank for Wikisim project, it has to be fast enough that can run in real time on relatively small graphs. I started from optimizing the networkx, however, I found a very nice algorithm by **Cleve Mole** which takes the full advantage of sparse matrix operations. 
I implemented two versions of the algorithm in Python, both inspired  by the sparse fast solutions given in [**Cleve Moler**](https://en.wikipedia.org/wiki/Cleve_Moler)'s book, [*Experiments with MATLAB*](http://www.mathworks.com/moler/index_ncm.html). The power method is much faster with enough precision for our task. Our benchmarsk shows that this implementation is **faster than both Networkx and iGraph** implementationa magnititude of times.

## Personalized Pagerank
I modified the algorithm a little bit to be able to calculate **personalized Pagerank** as well. 

## Input Format
The input is a 2d array, each row of the array is an edge of the graph $[[a,b], [c,d]]$, $a$ and $b$ are the node numbers. The **personalization vector** is probability distribution over the nodes, a.k.a **teleporting vector**.

## Comparison with Popular Python Implementations: Networkx and iGraph
Both of the implementation (Exact Solution and PowerMethod) are much faster than their correspondent method in NetworkX. 

#

In [ ]:
#%load_ext pycodestyle_magic

In [89]:
#%%writefile ../src/pagerank.py
"""Two "fast" implementations of PageRank.

Pythom implementations of Matlab original in:
Cleve Moler, Experiments with MATLAB.
"""
# uncomment
from __future__ import division

import scipy as sp
import scipy.sparse as sprs
import scipy.spatial
import scipy.sparse.linalg

__author__ = "Armin Sajadi"
__copyright__ = "Copyright 215, The Wikisim Project"
__credits__ = ["Armin Sajadi"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Armin Sajadi"
__email__ = "sajadi@cs.dal.ca"
__status__ = "Development"


def pagerank(G, p=0.85,
                          personalize=None, reverse=False):
    """ Calculates pagerank given a csr graph

    Inputs:
    -------

    G: a csr graph.
    p: damping factor
    personlize: if not None, should be an array with the size of the nodes
                containing probability distributions.
                It will be normalized automatically
    reverse: If true, returns the reversed-pagerank

    outputs
    -------

    Pagerank Scores for the nodes

    """
    # In Moler's algorithm, $G_{ij}$ represents the existences of an edge
    # from node $j$ to $i$, while we have assumed the opposite!
    if not reverse:
        G = G.T

    n, _ = G.shape
    c = sp.asarray(G.sum(axis=0)).reshape(-1)

    k = c.nonzero()[0]

    D = sprs.csr_matrix((1/c[k], (k, k)), shape=(n, n))

    if personalize is None:
        personalize = sp.ones(n)
    personalize = personalize.reshape(n, 1)
    e = (personalize/personalize.sum())*n

    I = sprs.eye(n)
    x = sprs.linalg.spsolve((I - p*G.dot(D)), e)

    x = x / x.sum()
    return x


def pagerank_power(G, p=0.85, max_iter=100,
                                tol=1e-06, personalize=None, reverse=False):
    """ Calculates pagerank given a csr graph

    Inputs:
    -------
    G: a csr graph.
    p: damping factor
    max_iter: maximum number of iterations
    personlize: if not None, should be an array with the size of the nodes
                containing probability distributions.
                It will be normalized automatically.
    reverse: If true, returns the reversed-pagerank

    Returns:
    --------
    Pagerank Scores for the nodes

    """
    # In Moler's algorithm, $G_{ij}$ represents the existences of an edge
    # from node $j$ to $i$, while we have assumed the opposite!
    if not reverse:
        G = G.T

    n, _ = G.shape
    c = sp.asarray(G.sum(axis=0)).reshape(-1)

    k = c.nonzero()[0]

    D = sprs.csr_matrix((1/c[k], (k, k)), shape=(n, n))

    if personalize is None:
        personalize = sp.ones(n)
    personalize = personalize.reshape(n, 1)
    e = (personalize / personalize.sum())*n

    z = (((1-p) * (c != 0) + (c == 0)) / n)[sp.newaxis, :]
    G = p * G.dot(D)

    x = e / n
    oldx = sp.zeros((n, 1))

    iteration = 0

    while sp.linalg.norm(x-oldx) > tol:
        oldx = x
        x = G.dot(x) + e.dot(z.dot(x))
        iteration += 1
        if iteration >= max_iter:
            break
    x = x / sum(x)

    return x.reshape(-1)


# Testing the algorithm

In [114]:
# %%writefile ../test/pagerank_test.py

import networkx as nx
import random
import timeit
import numpy as np
#import igraph
from numpy.testing import assert_array_almost_equal
import os
import sys
#current = os.path.dirname(os.path.realpath(__file__))
#src_path = os.path.join(current, '..')
#sys.path.insert(0,src_path)

#from src.pagerank import *

import unittest

class TestMolerPageRank(unittest.TestCase):
    def setUp(self):
        self.number_of_tests = 10
    def test_pagerank_1(self):
        n= 5
        rows = [0,1,2,2,2,3,3,4,4,4]
        cols = [1,2,1,3,4,0,2,0,2,3]
        data = [0.4923,0.0999,0.2132,0.0178,0.5694,0.0406,0.2047,0.861 ,0.3849,0.4829]

        p = 0.83
        perosonalize = sp.array([0.6005,0.1221,0.2542,0.4778,0.4275])
        G = sp.sparse.csr_matrix((data,(rows,cols)), shape=(n,n));
        pr = sp.array([0.1592,0.2114,0.3085,0.1   ,0.2208])
        
        calculated_pagerank = pagerank(G, p=p,
                          personalize=perosonalize)
        assert_array_almost_equal(calculated_pagerank,  pr, decimal = 4)
        
#     def test_exact_pagerank(self):
#         damping_factor = 0.85
#         for i in range(self.number_of_tests):
#             nxG, A, iG, customization_vector, customization_dict = get_random_graph()

#             Xnx  = nx.pagerank_numpy(nxG, alpha=damping_factor, personalization = customization_dict) 
#             Xnx =  np.array([v for k,v in Xnx.items() ])

#             Xml =  moler_pagerank(A, p=damping_factor, personalize=customization_vector)

#             assert_array_almost_equal(Xnx,  Xml, decimal = 5)
        
#     def test_power_pagerank(self):
#         damping_factor = 0.85
#         tol = 1e-06
#         for i in range(self.number_of_tests):
#             nxG, A, iG, customization_vector, customization_dict = get_random_graph()

#             Ynx =  nx.pagerank_scipy(nxG, alpha=damping_factor, tol=tol, personalization=customization_dict)
#             Ynx =  np.array([v for k,v in Ynx.items() ])

#             Yml =  moler_pagerank_power(A, p=damping_factor, tol=tol, personalize=customization_vector)


#             assert_array_almost_equal(Ynx,  Yml, decimal = 5)
            
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    #unittest.main()

/anaconda3/envs/general/lib/python3.7/site-packages/numpy/matrixlib/defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)
.
----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


In [109]:
n= 5
rows = [0,1,2,2,2,3,3,4,4,4]
cols = [1,2,1,3,4,0,2,0,2,3]
data = [0.4923,0.0999,0.2132,0.0178,0.5694,0.0406,0.2047,0.861 ,0.3849,0.4829]

p = 0.83
perosonalize = sp.array([0.6005,0.1221,0.2542,0.4778,0.4275])
G = sp.sparse.csr_matrix((data,(rows,cols)), shape=(n,n));
pr = sp.array([0.2618,0.1203,0.1475,0.154 ,0.3163])

calculated_pagerank = pagerank(G, p=p,
                  personalize=perosonalize)
print(calculated_pagerank)

G = nx.from_scipy_sparse_matrix(G, create_using=nx.DiGraph())
customization_dict = dict(enumerate(perosonalize.reshape(-1)))

pr = nx.pagerank_numpy(G, alpha=p, personalization = customization_dict) 
print("sp.array(%s)" % (sp.array2string(sp.array(list(pr.values())), precision=6, separator=','),))



[0.15924678 0.21141255 0.3085205  0.10003821 0.22078196]
sp.array([0.159247,0.211413,0.308521,0.100038,0.220782])


In [34]:
import scipy as sp
import networkx as nx

#sp.set_printoptions(precision=4)
n=10
m=10
nxG = nx.gnm_random_graph(n=n, m=m, directed=True)
for e in nxG.edges():
     nxG[e[0]][e[1]]['weight']=sp.rand()
customization_vector = sp.random.random(n)   
customization_dict = dict(enumerate(customization_vector.reshape(-1)))
A=nx.to_scipy_sparse_matrix(nxG)        
alpha = (0.5) * sp.random.random_sample() + 0.5
pr = nx.pagerank_numpy(nxG, alpha=alpha, personalization = customization_dict) 
rows, cols = A.nonzero()
print("n=", n)
print("rows =", sp.array2string(rows, separator=','))
print("cols =", sp.array2string(cols, separator=','))
print("data =", sp.array2string(A.data, precision=4, separator=','))
print ("alpha = %.2f" % (alpha, ))
# print ("custom_vector =", sp.array2string(customization_vector, precision=4, separator=','))
# print ("pagerank =", sp.array2string(sp.array(list(pr.values())), precision=4, separator=','))


n= 10
rows = [2,2,4,5,5,5,6,6,9,9]
col = [4,5,5,3,4,9,1,2,2,4]
data = [0.4565,0.2861,0.573 ,0.0025,0.4829,0.3866,0.3041,0.3407,0.2653,0.8079]
alpha = 0.92


In [47]:
n=5
customization_vector = sp.random.random(n)   
print ("custom_vector =", sp.array2string(customization_vector, precision=4, separator=','))
#print ("pagerank =", sp.array2string(sp.array(list(pr.values())), precision=4, separator=','))


custom_vector = [0.3325,0.2649,0.8656,0.737 ,0.4814]


In [96]:
n= 5
rows = [0,1,2,2,2,3,3,4,4,4]
cols = [1,2,1,3,4,0,2,0,2,3]
data = [0.4923,0.0999,0.2132,0.0178,0.5694,0.0406,0.2047,0.861 ,0.3849,0.4829]

alpha1 = 0.83
custom_vector1 = sp.array([0.6005,0.1221,0.2542,0.4778,0.4275])
G1 = sp.sparse.csr_matrix((data,(rows,cols)), shape=(n,n));
pr1 = sp.array([0.1592,0.2114,0.3085,0.1   ,0.2208])

n= 10
rows = [2,2,4,5,5,5,6,6,9,9]
cols = [4,5,5,3,4,9,1,2,2,4]
data = [0.4565,0.2861,0.573 ,0.0025,0.4829,0.3866,0.3041,0.3407,0.2653,0.8079]

custom_vector2 = sp.array([0.8887,0.6491,0.7843,0.7103,0.7428,0.6632,0.7351,0.3006,0.8722,0.1652])
alpha2 = 0.92
G2 = sp.sparse.csr_matrix((data,(rows,cols)), shape=(n,n));
pr2=sp.array([0.0315,0.0714,0.1831,0.0254,0.2057,0.1589,0.1237,0.0106,0.0309,0.1587])

n=5
rows = [2]
cols = [4]
data = [0.5441]

custom_vector3 = sp.array([0.0884,0.2797,0.3093,0.5533,0.985 ])
alpha3 = 0.81
G3 = sp.sparse.csr_matrix((data,(rows,cols)), shape=(n,n));
pr3=sp.array([0.0104,0.0328,0.4244,0.0648,0.4677])

n=5
rows = []
cols = []
data = []

custom_vector4 = sp.array([0.2534,0.8945,0.9562,0.056 ,0.9439])
alpha4 = 0.70
G4 = sp.sparse.csr_matrix((data,(rows,cols)), shape=(n,n));
pr4 = sp.array([0.0816,0.2882,0.3081,0.018 ,0.3041])

n=0
rows = []
cols = []
data = []

custom_vector5 = sp.array([0.3325,0.2649,0.8656,0.737 ,0.4814])
alpha5 = 0.70
G5 = sp.sparse.csr_matrix((data,(rows,cols)), shape=(n,n));
pr5 = sp.array([])

In [113]:
G = nx.from_scipy_sparse_matrix(G1, create_using=nx.DiGraph())
customization_dict = dict(enumerate(custom_vector1.reshape(-1)))

pr = nx.pagerank_numpy(G, alpha=alpha1, personalization = customization_dict) 
print("sp.array(%s)" % (sp.array2string(sp.array(list(pr.values())), precision=4, separator=','),))

sp.array([0.1592,0.2114,0.3085,0.1   ,0.2208])


In [ ]:
!python  ../test/pagerank_test.py

In [ ]:
!pip install python-igraph

# Benchmarking

To avoid the clutter, we only visualize the fastest method from each implementation, that is: 
.

In [ ]:
%%writefile benchmarking.py
import scipy as sp
import timeit
import sys
import networkx as nx
from src.pagerank import moler_pagerank
from src.pagerank import moler_pagerank_power
from test.pagerank_test import get_random_graph

sys.path.insert(0, '..')

n = 5
number_of_graphs = 20

size_vector = sp.zeros(number_of_graphs)
netx_pagerank_times = sp.zeros(number_of_graphs)
netx_pagerank_times_numpy = sp.zeros(number_of_graphs)
netx_pagerank_times_scipy = sp.zeros(number_of_graphs)
moler_pagerank_times = sp.zeros(number_of_graphs)
moler_pagerank_times_power = sp.zeros(number_of_graphs)
ig_pagerank_times = sp.zeros(number_of_graphs)

damping_factor = 0.85
tol = 1e-3


for i in range(number_of_graphs):
    nxG, A, iG, customization_vector, customization_dict = get_random_graph(
        min_size=100, max_size=1000)
    size_vector[i] = nxG.number_of_edges()

    netx_pagerank_times[i] = timeit.timeit(
        lambda: nx.pagerank(
            nxG,
            alpha=damping_factor,
            tol=tol),
        number=n) / n
    netx_pagerank_times_numpy[i] = timeit.timeit(
        lambda: nx.pagerank_numpy(
            nxG, alpha=damping_factor), number=n) / n
    netx_pagerank_times_scipy[i] = timeit.timeit(
        lambda: nx.pagerank_scipy(
            nxG, alpha=damping_factor, tol=tol), number=n) / n

    ig_pagerank_times[i] = timeit.timeit(
        lambda: iG.personalized_pagerank(
            directed=True,
            damping=damping_factor,
            weights=iG.es['weight'],
            implementation="prpack"),
        number=n) / n

    moler_pagerank_times[i] = timeit.timeit(
        lambda: moler_pagerank(
            A, p=damping_factor), number=n) / n
    moler_pagerank_times_power[i] = timeit.timeit(
        lambda: moler_pagerank_power(
            A, p=damping_factor, tol=tol), number=n) / n


argsort = size_vector.argsort()

size_vector_sorted = size_vector[argsort]

netx_pagerank_times_sorted = netx_pagerank_times[argsort]
netx_pagerank_times_numpy_sorted = netx_pagerank_times_numpy[argsort]
netx_pagerank_times_scipy_sorted = netx_pagerank_times_scipy[argsort]

moler_pagerank_times_sorted = moler_pagerank_times[argsort]
moler_pagerank_times_power_sorted = moler_pagerank_times_power[argsort]

ig_pagerank_times_sorted = ig_pagerank_times[argsort]


print("Done")


In [ ]:
%%writefile plotting.py

import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(num=None, figsize=(7, 5), dpi=80, facecolor='w', edgecolor='k')





#plt.plot(size_vector_sorted, netx_pagerank_times_sorted, 'o-',  ms=8, lw=2,alpha=0.7, color='cyan', label='networkx.PageRank')
#plt.plot(size_vector_sorted, netx_pagerank_times_numpy_sorted, 'v-', ms=8, lw=2,alpha=0.7, color='magenta', label='networkx.PageRank_numpy')
plt.plot(size_vector_sorted, netx_pagerank_times_scipy_sorted, 'P-', ms=8, lw=2,alpha=0.7, color='blue', label='networkx.PageRank_scipy')

plt.plot(size_vector_sorted, ig_pagerank_sorted, 'x-', ms=8, lw=2,alpha=0.7, color='black', label='iGraph_PageRank_ARPACK')

plt.plot(size_vector_sorted, moler_pagerank_times, '*-', ms=8, lw=2,alpha=0.7, color='red', label='moler_pagerank_times')
plt.plot(size_vector_sorted, moler_pagerank_times_power, '^-', ms=8, lw=2,alpha=0.7, color='green', label='moler_pagerank_times_Power')


plt.xlabel('Number of the edges')
plt.ylabel('Time (Seconds)')


plt.tight_layout()
plt.legend(loc=2)
plt.savefig('pagerank_exact.eps')
plt.show()


# Comparing Approximation Methods (Power Methods)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(num=None, figsize=(7, 5), dpi=80, facecolor='w', edgecolor='k')

argsort = size_vector.argsort()

size_vector_sorted = size_vector[argsort]
netx_pagerank_times_scipy_sorted = netx_pagerank_times_scipy[argsort]
moler_pagerank_times_power_sorted = moler_pagerank_times_power[argsort]



plt.plot(size_vector_sorted, netx_pagerank_times_scipy_sorted, 'P-', ms=8, lw=2,alpha=0.7, color='black', label='networkx.PageRank_scipy')
plt.plot(size_vector_sorted, moler_pagerank_times_power, '^-', ms=8, lw=2,alpha=0.7, color='green', label='moler_pagerank_times_Power')
#plt.plot(size_vector_sorted, ig_pagerank, '^-', ms=8, lw=2,alpha=0.7, color='red', label='moler_pagerank_times_Power')

plt.xlabel('Number of the edges')
plt.ylabel('Time (Seconds)')


plt.tight_layout()
plt.legend(loc=2)
plt.savefig('pagerank.eps')
plt.show()
